In [1]:
import pandas as pd
import os
import re
import emoji
from nltk import word_tokenize
from nltk import PorterStemmer

In [2]:
path = 'news_datasets'
csv_files = os.listdir(path)
dfs=[]
for filename in csv_files:
    df = pd.read_csv(path + '/' + filename)
    dfs.append(df)
real_news_df = pd.concat(dfs, ignore_index=True)
real_news_df.rename(columns={'Title': 'title', 'Description': 'text', 'Label': 'label'}, inplace=True)

In [3]:
real_news_df.shape

(14261, 3)

In [4]:
real_news_df.isnull().sum()

title    0
text     1
label    0
dtype: int64

In [5]:
real_news_df.dropna(inplace=True)

In [6]:
real_news_df.shape

(14260, 3)

In [7]:
real_news_df.drop_duplicates(subset='text', inplace=True)
real_news_df.reset_index(drop=True, inplace=True)

In [8]:
real_news_df.shape

(12411, 3)

In [9]:
real_news_df.dtypes

title    object
text     object
label     int64
dtype: object

In [10]:
fake_news_df = pd.read_csv('WELFake_Dataset.csv',index_col=0)
fake_news_df = fake_news_df[(fake_news_df['label'] == 1) & (fake_news_df['text'].str.strip() != '')]
fake_news_df['label'] = 0

In [11]:
fake_news_df.shape

(36363, 3)

In [12]:
fake_news_df.isnull().sum()

title    558
text      39
label      0
dtype: int64

In [13]:
fake_news_df.dropna(inplace=True)

In [14]:
fake_news_df.shape

(35766, 3)

In [15]:
fake_news_df.drop_duplicates(subset='text', inplace=True)
fake_news_df.reset_index(drop=True, inplace=True)

In [16]:
fake_news_df.shape

(27577, 3)

In [17]:
fake_news_df.dtypes

title    object
text     object
label     int64
dtype: object

In [18]:
fake_news_df = fake_news_df.head(12376)
fake_news_df.shape

(12376, 3)

In [19]:
final_news_df = pd.concat([real_news_df,fake_news_df],ignore_index=True)
final_news_df.shape

(24787, 3)

In [20]:
final_news_df.describe()

,label
count,24787.000000
mean,0.500706
std,0.500010
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [21]:
final_news_df.to_csv('final_news_dataset.csv',index=False)

In [22]:
contractions_dict = {"ain't": "are not", "aren't": "are not", "It’s": "it is",
                     "can't": "cannot", "can't've": "cannot have",
                     "'cause": "because", "could've": "could have", "couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not",
                     "don't": "do not", "hadn't": "had not", "hadn't've": "had not have",
                     "hasn't": "has not", "haven't": "have not", "he'd": "he would",
                     "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did", "how'd'y": "how do you", "how'll": "how will",
                     "i'd": "i would", "i'd've": "i would have", "i'll": "i will",
                     "i'll've": "i will have", "i'm": "i am", "I've": "i have", "isn't": "is not",
                     "it'd": "it would", "it'd've": "it would have", "it'll": "it will",
                     "it'll've": "it will have", "let's": "let us", "ma'am": "madam",
                     "mayn't": "may not", "might've": "might have", "mightn't": "might not",
                     "mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not",
                     "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
                     "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have", "should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                     "that'd": "that would", "that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have", "they'll": "they will",
                     "they'll've": "they will have", "they're": "they are", "they've": "they have",
                     "to've": "to have", "wasn't": "was not", "we'd": "we would",
                     "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                     "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will",
                     "what'll've": "what will have", "what're": "what are", "what've": "what have",
                     "when've": "when have", "where'd": "where did", "where've": "where have",
                     "who'll": "who will", "who'll've": "who will have", "who've": "who have",
                     "why've": "why have", "will've": "will have", "won't": "will not",
                     "won't've": "will not have", "would've": "would have", "wouldn't": "would not",
                     "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have", "y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would", "you'd've": "you would have",
                     "you'll": "you will", "you'll've": "you will have", "you're": "you are"}


def correct_words(words):
    corrected_words = []
    for word in words:
        key = word.replace("’", "'")
        if key in contractions_dict.keys():
            corrected_words.append(contractions_dict[key])
        elif word in contractions_dict.keys():
            corrected_words.append(contractions_dict[word])
        else:
            corrected_words.append(word.replace("’s", " is").replace("'s", " is"))
    return corrected_words

In [23]:
stemmer = PorterStemmer()
def process_text(text):
    text = ''.join([emoji.demojize(i, delimiters=(' ', ' ')) for i in text])
    words = text.strip().split()
    words = [word.lower() for word in words]
    words = [re.sub(r"[^a-z\s’'#@*0-9]+", '', word) for word in words]
    words = correct_words(words)
    text = ' '.join([re.sub(r"[’']", '', word) for word in words])
    text = re.sub(r'http\S+', '', text)
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

In [24]:
final_news_df['text'] = final_news_df['title'] + ' ' + final_news_df['text']
final_news_df['text'] = final_news_df['text'].apply(process_text)
final_news_df.head(10)

,title,text,label
0,Voice referendum: Australia to hold historic I...,voic referendum australia to hold histor indig...,1
1,Ukraine war: 'Drone attack' hits airport in no...,ukrain war drone attack hit airport in northwe...,1
2,James Cleverly visits Beijing as MPs criticise...,jame cleverli visit beij as mp criticis china ...,1
3,Nagorno-Karabakh: 'People are fainting queuing...,nagornokarabakh peopl are faint queu up for br...,1
4,Hurricane Idalia: What is a storm surge and wh...,hurrican idalia what is a storm surg and whi i...,1
5,On patrol with Tunisia's coast guard: Nine boa...,on patrol with tunisia is coast guard nine boa...,1
6,Hurricane Idalia: Florida hunkers down for 'un...,hurrican idalia florida hunker down for unprec...,1
7,Little boy among three killed in County Tipper...,littl boy among three kill in counti tipperari...,1
8,Harrison Floyd: Bond agreement reached for jai...,harrison floyd bond agreement reach for jail g...,1
9,Rare footage of St Elmo's Fire as hurricane ap...,rare footag of st elmo is fire as hurrican app...,1


In [25]:
final_news_df.isnull().sum()

title    0
text     0
label    0
dtype: int64

In [26]:
final_news_df.to_csv('final_cleaned_news_dataset.csv', index=False)